<a href="https://colab.research.google.com/github/Edakksm/Deep-Learning/blob/master/BERT_Smitha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pytorch-pretrained-bert pytorch-nlp

import tensorflow as tf
import torch
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertForSequenceClassification, BertAdam, BertConfig, BertTokenizer
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
device_name = tf.test.gpu_device_name
print(device_name)
if device_name != '/device:GPU:0':
  print("GPU Not found")
else:
  print("GPU Found")

<function gpu_device_name at 0x7f11d21f7510>
GPU Not found


In [0]:
df = pd.read_csv('in_domain_train.tsv', delimiter='\t', header=None, names=['sentence_source','label','label_notes','sentence'])

In [6]:
df.head()

,sentence_source,label,label_notes,sentence
0,gj04,1,NaN,"Our friends won't buy this analysis, let alone..."
1,gj04,1,NaN,One more pseudo generalization and I'm giving up.
2,gj04,1,NaN,One more pseudo generalization or I'm giving up.
3,gj04,1,NaN,"The more we study verbs, the crazier they get."
4,gj04,1,NaN,Day by day the facts are getting murkier.


In [7]:
df.shape

(8551, 4)

In [8]:
df.sample(10)

,sentence_source,label,label_notes,sentence
6443,d_98,0,*,Mary picked any of the flowers.
5520,b_73,0,*,I've never seen the one taller man than my fat...
990,bc01,1,NaN,The testing of such drugs on oneself is too ri...
2868,l-93,1,NaN,That chisel carved the statue.
6671,m_02,1,NaN,What did the wealthy young man buy for his sec...
2471,l-93,1,NaN,Sarah sang.
6150,c_13,1,NaN,I sent Louis a book.
7561,sks13,1,NaN,the girls likes them.
8427,ad03,0,*,The dragons were slain all.
1620,r-67,1,NaN,"Swimming is fun, and I believe that people who..."


In [0]:
sentences = df.sentence

In [10]:
sentences

0       Our friends won't buy this analysis, let alone...
1       One more pseudo generalization and I'm giving up.
2        One more pseudo generalization or I'm giving up.
3          The more we study verbs, the crazier they get.
4               Day by day the facts are getting murkier.
                              ...                        
8546                     Poseidon appears to own a dragon
8547                       Digitize is my happiest memory
8548                       It is easy to slay the Gorgon.
8549         I had the strangest feeling that I knew you.
8550                  What all did you get for Christmas?
Name: sentence, Length: 8551, dtype: object

In [0]:
sentences = [ "[CLS] " + sentence + " [SEP]" for sentence in sentences]

In [15]:
sentences[0]

"[CLS] Our friends won't buy this analysis, let alone the next one we propose. [SEP]"

In [0]:
# Import BERT Tokenizer

In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
tokenized_texts[0]

100%|██████████| 231508/231508 [00:00<00:00, 2694090.66B/s]


['[CLS]',
 'our',
 'friends',
 'won',
 "'",
 't',
 'buy',
 'this',
 'analysis',
 ',',
 'let',
 'alone',
 'the',
 'next',
 'one',
 'we',
 'propose',
 '.',
 '[SEP]']

In [18]:
print(tokenized_texts[0])

['[CLS]', 'our', 'friends', 'won', "'", 't', 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.', '[SEP]']


In [0]:
# Pad sequences

In [0]:
MAX_LEN = 128

In [0]:
input_ids = [ tokenizer.convert_tokens_to_ids(token) for token in tokenized_texts]

In [23]:
print(input_ids[0])

[101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102]


In [24]:
len(input_ids[0])

19

In [25]:
len(input_ids[1])

14

In [0]:
input_ids = pad_sequences(input_ids, MAX_LEN, padding="post", truncating="post", dtype="long")

In [27]:
len(input_ids[0])

128

In [28]:
print(input_ids[0])

[  101  2256  2814  2180  1005  1056  4965  2023  4106  1010  2292  2894
  1996  2279  2028  2057 16599  1012   102     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


In [0]:
attention_masks =[]
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [31]:
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [0]:
# split into train and validation sets
labels = df.label.values
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2018, test_size=0.1)

In [35]:
train_inputs.shape

(7695, 128)